# Transformer 실습

이번 실습에서는 감정 분석 task에 RNN 대신 Transformer를 구현하여 적용해 볼 것입니다.
Library import나 dataloader 생성은 RNN 실습 때와 똑같기 때문에 설명은 넘어가도록 하겠습니다.

### LongTensor
torch.LongTensor를 사용하는 이유는 다음과 같습니다

토큰 ID의 데이터 타입: 토큰 ID는 정수(양의 정수)이며, PyTorch에서 정수형 데이터를 표현하기 위해 LongTensor(64비트 정수) 또는 IntTensor(32비트 정수)가 필요합니다.

어휘 사전 크기 지원: LongTensor(int64)는 IntTensor(int32)보다 더 넓은 범위의 정수를 표현할 수 있어, 큰 어휘 사전을 가진 토크나이저에서도 문제없이 사용할 수 있습니다.

패딩 및 특수 토큰 지원: 패딩 토큰 ID와 같은 특수 토큰도 정수로 표현되며, 모델이 이를 올바르게 처리하려면 모든 입력이 동일한 타입(LongTensor)이어야 합니다.

간단히 말해, 텍스트를 숫자(토큰 ID)로 변환한 후 PyTorch가 이해할 수 있는 텐서 형식으로 바꿔주는 역할을 합니다.

In [1]:
import torch
import os
os.environ['KMP_DUPLICATE_LIB_OK']='TRUE'
os.environ['CUDA_LAUNCH_BLOCKING']='1'
torch.cuda.empty_cache()

from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


# ds = load_dataset("stanfordnlp/imdb")

wikitext = load_dataset("wikitext", "wikitext-2-raw-v1")
train_ds = wikitext["train"]
test_ds = wikitext["test"]

# train_ds = load_dataset("stanfordnlp/imdb", split="train[:5%]")
# test_ds = load_dataset("stanfordnlp/imdb", split="test[:5%]")

tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')
max_len = 400



# def collate_fn(batch):
 
#   texts, labels = [], []
#   for row in batch:
#     labels.append(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[-3])
#     texts.append(torch.LongTensor(tokenizer(row['text'], truncation=True, max_length=max_len).input_ids[:-3]))

#   texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
#   labels = torch.LongTensor(labels)

#   return texts, labels


# train_loader = DataLoader(
#     train_ds, batch_size=64, shuffle=True, collate_fn=collate_fn
# )
# test_loader = DataLoader(
#     test_ds, batch_size=64, shuffle=False, collate_fn=collate_fn
# )


def collate_fn(batch):
  texts, labels = [], []
  for row in batch:
    tokens = tokenizer(row['text'], truncation=True, max_length=max_len).input_ids
    # 토큰 길이 확인하여 충분한 길이인 경우만 처리
    if len(tokens) >= 3:  # 최소 3개 이상의 토큰이 있는지 확인
      labels.append(tokens[-3])
      texts.append(torch.LongTensor(tokens[:-3]))
    else:
      # 토큰이 충분하지 않은 경우 PAD 토큰으로 대체
      # 또는 이 샘플을 건너뛸 수도 있음
      if len(tokens) > 0:  # 최소 1개 이상 토큰이 있으면
        labels.append(tokenizer.pad_token_id)  # PAD 토큰을 라벨로 사용
        texts.append(torch.LongTensor(tokens))
      else:
        # 완전히 빈 토큰의 경우 건너뛰기
        continue
  
  if not texts:  # 모든 샘플이 필터링된 경우
    # 더미 데이터 반환 또는 None 반환
    return None
  
  texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
  labels = torch.LongTensor(labels)
  
  return texts, labels

# 데이터 로더 수정 (배치 크기 줄임)
train_loader = DataLoader(
    train_ds, batch_size=32, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    test_ds, batch_size=32, shuffle=False, collate_fn=collate_fn
)
# None을 반환할 가능성 처리
def safe_collate(batch):
    result = collate_fn(batch)
    if result is None:
        # 대체 배치 생성
        dummy_text = torch.ones(1, 10, dtype=torch.long) * tokenizer.pad_token_id
        dummy_label = torch.zeros(1, dtype=torch.long)
        return dummy_text, dummy_label
    return result

train_loader = DataLoader(
    train_ds, batch_size=32, shuffle=True, collate_fn=safe_collate
)
test_loader = DataLoader(
    test_ds, batch_size=32, shuffle=False, collate_fn=safe_collate
)

Using cache found in C:\Users\junu/.cache\torch\hub\huggingface_pytorch-transformers_main


In [2]:
tokenizer.model_max_length

512

In [3]:
# 토크나이저 전체 어휘 확인
vocab_size = len(tokenizer)
print(f"🔍 토크나이저 어휘 전체 크기: {vocab_size}개 토큰")

# 전체 ID 범위
print(f"🔢 토큰 ID 범위: 0 ~ {vocab_size-1}")

# 특수 토큰 정보
special_tokens = {
    'PAD': tokenizer.pad_token_id,
    'UNK': tokenizer.unk_token_id,
    'SEP': tokenizer.sep_token_id, 
    'CLS': tokenizer.cls_token_id,
    'MASK': tokenizer.mask_token_id if hasattr(tokenizer, 'mask_token_id') else None
}

print("\n🔑 특수 토큰:")
for name, token_id in special_tokens.items():
    if token_id is not None:
        token = tokenizer.convert_ids_to_tokens([token_id])[0]
        print(f"{name}: ID {token_id}, 토큰 '{token}'")

# 토큰 타입별 분류
def token_type(token):
    if token.startswith('[') and token.endswith(']'):
        return "특수토큰"
    elif token.startswith('##'):
        return "단어조각"
    elif token.isalpha():
        return "단어"
    elif token.isdigit():
        return "숫자"
    elif token.isalnum():
        return "영숫자"
    else:
        return "기타"

# 전체 어휘 분석 (샘플링 방식)
from collections import Counter
import random
print("\n📊 전체 어휘 분석 (샘플링 기반):")
sample_size = min(5000, vocab_size)  # 샘플 크기 (최대 5000개)
sample_ids = random.sample(range(vocab_size), sample_size)
token_types = Counter()

for token_id in sample_ids:
    token = tokenizer.convert_ids_to_tokens([token_id])[0]
    token_types[token_type(token)] += 1

print(f"샘플링된 {sample_size}개 토큰 기준 타입 분포:")
for type_name, count in token_types.most_common():
    print(f"- {type_name}: {count}개 ({count/sample_size*100:.1f}%)")

# 실제로 어떤 토큰들이 있는지 확인 (처음 20개, 마지막 20개, 중간 20개)
print("\n📝 어휘 샘플 (처음/중간/마지막 일부):")

print("\n처음 20개 토큰:")
for i in range(20):
    token = tokenizer.convert_ids_to_tokens([i])[0]
    print(f"ID {i:5d}: '{token}'")

mid_point = vocab_size // 2
print(f"\n중간 {mid_point} 근처 20개 토큰:")
for i in range(mid_point-10, mid_point+10):
    token = tokenizer.convert_ids_to_tokens([i])[0]
    print(f"ID {i:5d}: '{token}'")

print(f"\n마지막 20개 토큰:")
for i in range(vocab_size-20, vocab_size):
    token = tokenizer.convert_ids_to_tokens([i])[0]
    print(f"ID {i:5d}: '{token}'")

# 데이터셋에서 실제 사용되는 라벨 수
all_labels_set = set()
batch_count = 0

print("\n🏷️ 데이터셋 라벨 분석 중...")
for texts, labels in train_loader:
    all_labels_set.update(labels.tolist())
    batch_count += 1
    if batch_count >= 100:  # 100개 배치만 확인
        break

print(f"✅ 데이터셋에서 실제 사용되는 고유 라벨 수: {len(all_labels_set)}개")
print(f"✅ 총 어휘 대비 사용 비율: {len(all_labels_set)/vocab_size*100:.2f}%")

🔍 토크나이저 어휘 전체 크기: 30522개 토큰
🔢 토큰 ID 범위: 0 ~ 30521

🔑 특수 토큰:
PAD: ID 0, 토큰 '[PAD]'
UNK: ID 100, 토큰 '[UNK]'
SEP: ID 102, 토큰 '[SEP]'
CLS: ID 101, 토큰 '[CLS]'
MASK: ID 103, 토큰 '[MASK]'

📊 전체 어휘 분석 (샘플링 기반):
샘플링된 5000개 토큰 기준 타입 분포:
- 단어: 3636개 (72.7%)
- 단어조각: 950개 (19.0%)
- 특수토큰: 191개 (3.8%)
- 숫자: 168개 (3.4%)
- 기타: 43개 (0.9%)
- 영숫자: 12개 (0.2%)

📝 어휘 샘플 (처음/중간/마지막 일부):

처음 20개 토큰:
ID     0: '[PAD]'
ID     1: '[unused0]'
ID     2: '[unused1]'
ID     3: '[unused2]'
ID     4: '[unused3]'
ID     5: '[unused4]'
ID     6: '[unused5]'
ID     7: '[unused6]'
ID     8: '[unused7]'
ID     9: '[unused8]'
ID    10: '[unused9]'
ID    11: '[unused10]'
ID    12: '[unused11]'
ID    13: '[unused12]'
ID    14: '[unused13]'
ID    15: '[unused14]'
ID    16: '[unused15]'
ID    17: '[unused16]'
ID    18: '[unused17]'
ID    19: '[unused18]'

중간 15261 근처 20개 토큰:
ID 15251: 'perspectives'
ID 15252: 'reviewing'
ID 15253: 'mets'
ID 15254: 'commandant'
ID 15255: 'radial'
ID 15256: '##kha'
ID 15257: 'flashlight'
ID 15258: 

## Self-attention

이번에는 self-attention을 구현해보겠습니다.
Self-attention은 shape이 (B, S, D)인 embedding이 들어왔을 때 attention을 적용하여 새로운 representation을 만들어내는 module입니다.
여기서 B는 batch size, S는 sequence length, D는 embedding 차원입니다.
구현은 다음과 같습니다.

B: 배치 크기 (Batch size)  
S: 시퀀스 길이 (Sequence length) - 토큰의 개수  
D: 임베딩 차원 (Embedding dimension)  
H: 어텐션 헤드의 수 (Number of attention heads)  
D': 헤드 당 차원 (Dimension per head), D = H × D'  
멀티 헤드 어텐션에서는 하나의 큰 어텐션 연산 대신, 여러 개의 작은 어텐션 연산을 병렬로 수행합니다. 이를 위해 Q, K, V 행렬을 여러 헤드로 분할해야 합니다.
원래 Q, K, V의 형태가 (B, S, D)일 때:
이를 (B, S, H, D')로 변형합니다. 여기서 D' = D/H입니다.
각 헤드마다 독립적으로 어텐션을 계산합니다.
모든 헤드의 결과를 다시 합쳐서 (B, S, D) 형태로 만듭니다.
이 과정을 구현하려면 Q, K, V 행렬을 여러 헤드로 나누어야 하며, 이때 원래 차원 D가 헤드 수 H로 나누어 떨어져야 한다는 제약조건이 있습니다.

 D = H × D'  
 D' = D/H  
 D' 는 디 프라임이라고 부름

d_model 가 임베딩 모델의 dimension 임
동시에 입력 값이기도 함
빈 입력은 padding으로 처리됨  
padding 처리는 mask임


In [4]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
  def __init__(self, input_dim, d_model):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model

    self.wq = nn.Linear(input_dim, d_model)
    self.wk = nn.Linear(input_dim, d_model)
    self.wv = nn.Linear(input_dim, d_model)
    self.dense = nn.Linear(d_model, d_model)
  

    self.softmax = nn.Softmax(dim=-1)

  def forward(self, x, mask):
    # 입력 값을 q,k,v로 나눠서 가중치를 계산한다
    q, k, v = self.wq(x), self.wk(x), self.wv(x)
    
    # 행렬 곱으로 가중치에 가중치를 계산
    score = torch.matmul(q, k.transpose(-1, -2)) # (B, S, D) * (B, D, S) = (B, S, S)
    score = score / sqrt(self.d_model)

    if mask is not None:
      # 아주 작은 값으로
      score = score + (mask * -1e9)

    score = self.softmax(score)
    result = torch.matmul(score, v)
    result = self.dense(result)

    return result

대부분은 Transformer 챕터에서 배운 수식들을 그대로 구현한 것에 불과합니다.
차이점은 `mask`의 존재여부입니다.
이전 챕터에서 우리는 가변적인 text data들에 padding token을 붙여 하나의 matrix로 만든 방법을 배웠습니다.
실제 attention 계산에서는 이를 무시해주기 위해 mask를 만들어 제공해주게 됩니다.
여기서 mask의 shape은 (B, S, 1)로, 만약 `mask[i, j] = True`이면 그 변수는 padding token에 해당한다는 뜻입니다.
이러한 값들을 무시해주는 방법은 shape이 (B, S, S)인 `score`가 있을 때(수업에서 배운 $A$와 동일) `score[i, j]`에 아주 작은 값을 더해주면 됩니다. 아주 작은 값은 예를 들어 `-1000..00 = -1e9` 같은 것이 있습니다.
이렇게 작은 값을 더해주고 나면 softmax를 거쳤을 때 0에 가까워지기 때문에 weighted sum 과정에서 padding token에 해당하는 `v` 값들을 무시할 수 있게 됩니다.

다음은 self-attention과 feed-forward layer를 구현한 모습입니다.

In [5]:
class TransformerLayer(nn.Module):
  def __init__(self, input_dim, d_model, dff):
    super().__init__()

    self.input_dim = input_dim
    self.d_model = d_model
    self.dff = dff

    self.sa = SelfAttention(input_dim, d_model)
    self.ffn = nn.Sequential(
      nn.Linear(d_model, dff),
      nn.ReLU(),
      nn.Linear(dff, d_model)
    )

  def forward(self, x, mask):
    x = self.sa(x, mask)
    x = self.ffn(x)

    return x

보시다시피 self-attention의 구현이 어렵지, Transformer layer 하나 구현하는 것은 수업 때 다룬 그림과 크게 구분되지 않는다는 점을 알 수 있습니다.

## Positional encoding

이번에는 positional encoding을 구현합니다. Positional encoding의 식은 다음과 같습니다:
$$
\begin{align*} PE_{pos, 2i} &= \sin\left( \frac{pos}{10000^{2i/D}} \right), \\ PE_{pos, 2i+1} &= \cos\left( \frac{pos}{10000^{2i/D}} \right).\end{align*}
$$

이를 Numpy로 구현하여 PyTorch tensor로 변환한 모습은 다음과 같습니다:

In [6]:
# 모델 학습 전에 라벨 확인

label_min = float('inf')
label_max = float('-inf')
for i, (texts, labels) in enumerate(train_loader):
    # print(f"배치 {i+1}의 라벨 범위: {labels.min().item()} ~ {labels.max().item()}")
    batch_min = labels.min().item()
    batch_max = labels.max().item()
    
    label_min = min(label_min, batch_min)
    label_max = max(label_max, batch_max)
 

print(f"\n전체 확인된 라벨 범위: {label_min} ~ {label_max}")
print(f"필요한 num_classes: {label_max + 1}")  # 0부터 시작하므로 최대값+1이 필요한 클래스 수


전체 확인된 라벨 범위: 0 ~ 30006
필요한 num_classes: 30007


In [7]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)



print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


Positional encoding은 `angle_rads`를 구현하는 과정에서 모두 구현이 되었습니다. 여기서 `angle_rads`의 shape은 (S, D)입니다.
우리는 일반적으로 batch로 주어지는 shape이 (B, S, D)인 tensor를 다루기 때문에 마지막에 None을 활용하여 shape을 (1, S, D)로 바꿔주게됩니다.

위에서 구현한 `TransformerLayer`와 positional encoding을 모두 합친 모습은 다음과 같습니다:

In [8]:
class TextClassifier(nn.Module):
  def __init__(self, vocab_size, d_model, n_layers, dff, num_classes):
    super().__init__()

    # 모델 파라미터 저장
    self.vocab_size = vocab_size  # 어휘 사전 크기 (토큰 종류의 수)
    self.d_model = d_model        # 임베딩 및 모델 차원
    self.n_layers = n_layers      # 트랜스포머 레이어 수
    self.dff = dff                # 피드포워드 네트워크의 은닉층 차원
    self.num_classes = num_classes


    # 토큰 ID를 d_model 차원의 임베딩 벡터로 변환하는 레이어
    self.embedding = nn.Embedding(vocab_size, d_model)

    self.dropout = nn.Dropout(0.1)
    # 위치 인코딩 행렬 생성 (시퀀스 내 토큰 위치 정보 제공)
    # requires_grad=False로 설정하여 학습 중 이 파라미터는 업데이트되지 않음
    self.pos_encoding = nn.parameter.Parameter(positional_encoding(max_len, d_model), requires_grad=False)
    
    # n_layers 개수만큼 트랜스포머 레이어 생성
    self.layers = nn.ModuleList([TransformerLayer(d_model, d_model, dff) for _ in range(n_layers)])
    
    # 출력 차원을 클래스 수로 변경
    # self.classification = nn.Linear(d_model, num_classes)
        # 분류 헤드 개선
    self.classification = nn.Sequential(
        # nn.Linear(d_model, d_model // 2),
        nn.Linear(d_model, d_model ),
        nn.ReLU(),
        nn.Dropout(0.1),
        nn.Linear(d_model,num_classes)
    )

  def forward(self, x):
    # 패딩 마스크 생성: 패딩 토큰(pad_token_id)은 True, 나머지는 False
    mask = (x == tokenizer.pad_token_id)
    
    # 마스크 차원 추가: (batch_size, seq_len) -> (batch_size, 1, seq_len)
    # 어텐션 계산 시 브로드캐스팅을 위해 차원 확장
    mask = mask[:, None, :]
    
    # 현재 입력 시퀀스의 실제 길이
    seq_len = x.shape[1]

    # 토큰 ID를 임베딩 벡터로 변환
    x = self.embedding(x)
    
    # 임베딩에 sqrt(d_model) 곱하기 (트랜스포머 논문의 스케일링)
    x = x * sqrt(self.d_model)
    
    # 위치 인코딩 더하기 (현재 시퀀스 길이만큼만)
    x = x + self.pos_encoding[:, :seq_len]

    # 트랜스포머 레이어 통과
    for layer in self.layers:
      x = layer(x, mask)
      x = self.dropout(x) 
    # 첫 번째 토큰의 표현만 사용 (CLS 토큰 역할)
    x = x[:, 0]
    
    # 선형 레이어로 최종 분류 (시그모이드 함수는 loss 함수에 포함됨)
    x = self.classification(x)

    return x


# 모델 초기화: 
# - len(tokenizer): BERT 토크나이저의 어휘 사전 크기
# - 32: 임베딩 및 모델 차원 = d_model
# - 2: 트랜스포머 레이어 수
# - 32: 피드포워드 네트워크 차원
# - 30000: 라벨의 최대 텐서 크기
d_model = 256
n_layers = 5
dff = 4 * d_model

model = TextClassifier(len(tokenizer), d_model, n_layers, dff,200)

트랜스포머 레이어의 역할 : 어텐션을 분석해서 관계성을 찾는 것  
두번 하는 이유는 뭔가


트랜스포머도 결국 모델 구현체고 내부에서 피드포워드 네트워크 차원의 갯수를 지정하는데
특성을 분류하기 위한 숫자 라고 하기에 좀 적은 것 같다  


논문에선 차원 갯수 512에 레이어 4개였다

기존과 다른 점들은 다음과 같습니다:
1. `nn.ModuleList`를 사용하여 여러 layer의 구현을 쉽게 하였습니다.
2. Embedding, positional encoding, transformer layer를 거치고 난 후 마지막 label을 예측하기 위해 사용한 값은 `x[:, 0]`입니다. 기존의 RNN에서는 padding token을 제외한 마지막 token에 해당하는 representation을 사용한 것과 다릅니다. 이렇게 사용할 수 있는 이유는 attention 과정을 보시면 첫 번째 token에 대한 representation은 이후의 모든 token의 영향을 받습니다. 즉, 첫 번째 token 또한 전체 문장을 대변하는 의미를 가지고 있다고 할 수 있습니다. 그래서 일반적으로 Transformer를 text 분류에 사용할 때는 이와 같은 방식으로 구현됩니다.

## 학습

학습하는 코드는 기존 실습들과 동일하기 때문에 마지막 결과만 살펴보도록 하겠습니다.

In [9]:
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau

lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)  # 가중치 감쇠 추가
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)

In [10]:
# 언어 모델링에 맞는 정확도 계산 함수
def accuracy(model, dataloader):
  cnt = 0
  correct = 0
  top5_correct = 0

  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')

    with torch.no_grad():
      logits = model(inputs)
      
      # Top-1 정확도
      preds = torch.argmax(logits, dim=-1)
      correct += (preds == labels).sum().item()
      
      # Top-5 정확도
      _, top5_preds = torch.topk(logits, k=5, dim=-1)
      for i, label in enumerate(labels):
        if label in top5_preds[i]:
          top5_correct += 1
      
      cnt += labels.shape[0]

  return {
    'top1': correct / cnt,
    'top5': top5_correct / cnt
  }


In [11]:
# 학습 결과 저장
train_result = []
test_result = []
top5_train_result = []
top5_test_result = []

# 에폭 학습
n_epochs = 15  # 에포크 수

for epoch in range(n_epochs):
  total_loss = 0.
  model.train()
  for data in train_loader:
    model.zero_grad()
    inputs, labels = data
    inputs, labels = inputs.to('cuda'), labels.to('cuda')
    
    preds = model(inputs)
    loss = loss_fn(preds, labels)
    loss.backward()
    
    # 그래디언트 클리핑 추가
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()
    total_loss += loss.item()

  print(f"Epoch {epoch:3d} | Train Loss: {total_loss}")

  with torch.no_grad():
    model.eval()
    train_acc = accuracy(model, train_loader)
    test_acc = accuracy(model, test_loader)
    print(f"=========> Train acc: Top-1 {train_acc['top1']:.3f}, Top-5 {train_acc['top5']:.3f} | Test acc: Top-1 {test_acc['top1']:.3f}, Top-5 {test_acc['top5']:.3f}")
    
    # 결과 저장
    train_result.append(train_acc['top1'])
    test_result.append(test_acc['top1'])
    top5_train_result.append(train_acc['top5'])
    top5_test_result.append(test_acc['top5'])
    
    # 학습률 스케줄러 업데이트
    scheduler.step(test_acc['top1'])

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# 테스트 셋에서 하나의 샘플 선택하기
import random

# 모델을 평가 모드로 설정
model.eval()

# 테스트 셋에서 무작위로 샘플 하나 선택
sample_idx = random.randint(0, len(test_ds) - 1)
sample = test_ds[sample_idx]
original_text = sample['text']

# collate_fn과 동일한 방식으로 텍스트와 라벨 처리

tokenized = tokenizer(original_text, truncation=True, max_length=max_len).input_ids
print("🚀 ~ tokenized:", tokenized)
actual_label = tokenized[-3]  # collate_fn에서 사용한 방식과 동일하게 라벨 추출
input_text = torch.LongTensor(tokenized[:-3]).unsqueeze(0).to('cuda')  # 배치 차원 추가 및 GPU로 이동

actual_label_word = tokenizer.convert_ids_to_tokens([actual_label])[0]
input_words = tokenizer.convert_ids_to_tokens(tokenized[:-3])



# 모델 예측 수행
with torch.no_grad():
    output = model(input_text)
    predicted_label = torch.argmax(output, dim=-1).item()

# 결과 출력
print(f"원본 텍스트 (일부): {original_text[-64:]}...")
print(f"정답 라벨: {actual_label_word}")
print(f"입력 라벨: {input_words[-10:]}")
predicted_word = tokenizer.convert_ids_to_tokens([predicted_label,actual_label])
print(f"예측이 맞았는지 여부: {'맞음' if predicted_label == actual_label else '틀림'}")
print('예측/정답',predicted_word)

# 확률 분포 확인 (상위 5개)
probabilities = torch.nn.functional.softmax(output[0], dim=0)
top_probs, top_indices = torch.topk(probabilities, 5)

print("\n상위 5개 예측 확률:")
for i, (prob, idx) in enumerate(zip(top_probs.cpu().numpy(), top_indices.cpu().numpy())):
    print(f"예측 {i+1}: 라벨 {idx}, 확률 {prob:.4f}")

In [ ]:
import sys
# 또는 상대 경로
sys.path.append('..') 

from util.plot import plot_acc


plot_acc(train_result, test_result)